In [1]:
!git clone https://github.com/bwa2116/withPerformer.git

Cloning into 'withPerformer'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 124 (delta 75), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (124/124), 50.43 KiB | 3.36 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [2]:
cd /content/withPerformer

/content/withPerformer


In [3]:
# from vit_favor import ViTForClassfication
from vit import ViTForClassfication

import torch
from torch import nn, optim

import os

In [4]:
import matplotlib.pyplot as plt

In [5]:
from config import data_config
from data import load_data
from trainer import Trainer

In [6]:
torch.cuda.empty_cache()
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [7]:
exp_name = 'vit-with-20-epochs'
batch_size = 64 #256
epochs = 1 #100
lr = 1e-4
save_model_every = 0 #10


device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
data = 'Places365'

In [9]:
config = data_config[data]

# These are not hard constraints, but are used to prevent misconfigurations
assert config['hidden_size'] % config['num_attention_heads'] == 0
assert config['intermediate_size'] == 4 * config['hidden_size']
assert config['image_size'] % config['patch_size'] == 0

In [10]:
img_size = (config['image_size'], config['image_size'])
img_size

(64, 64)

In [ ]:
trainloader, testloader, _ = load_data(
    name=config['name'], img_size=img_size, batch_size=batch_size)

100%|██████████| 67498496/67498496 [00:02<00:00, 28146374.66it/s]


Extracting ./data/filelist_places365-standard.tar to ./data


100%|██████████| 26103685120/26103685120 [12:41<00:00, 34301533.45it/s]


Extracting ./data/train_256_places365standard.tar to ./data


100%|██████████| 525158400/525158400 [00:16<00:00, 32680767.79it/s]


Extracting ./data/val_256.tar to ./data


In [13]:
save_model_every_n_epochs = save_model_every

In [12]:
model = ViTForClassfication(config)

In [14]:
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-2)
loss_fn = nn.CrossEntropyLoss()

In [15]:
trainer = Trainer(
    model=model, config=config, optimizer=optimizer,
    loss_fn=loss_fn, exp_name=exp_name, device=device
)

In [ ]:
trainer.train(
    trainloader, testloader,
    epochs, save_model_every_n_epochs=save_model_every_n_epochs
)

Starting Epoch 1 of 1.


  1%|          | 154/28180 [00:18<55:03,  8.48batch/s]


KeyboardInterrupt: 

In [ ]:
print(device)

Performer

Starting Epoch 1 of 1.
100%|██████████| 938/938 [01:22<00:00, 11.35batch/s]
Epoch: 1, Train loss: 1.8025, Test loss: 1.4977, Accuracy: 0.3847

transformer

Starting Epoch 1 of 1.
100%|██████████| 938/938 [00:39<00:00, 23.76batch/s]
Epoch: 1, Train loss: 1.6264, Test loss: 1.0433, Accuracy: 0.5821


# Inspect

In [ ]:
from utils import visualize_images, visualize_attention, load_experiment

In [ ]:
visualize_images('MNIST') # Show some training images

In [ ]:
# Load Experiment
config, model, train_losses, test_losses, accuracies = load_experiment("vit-with-20-epochs")

# Create two subplots of train/test losses and accuracies
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
ax1.plot(train_losses, label="Train loss")
ax1.plot(test_losses, label="Test loss")
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss")
ax1.legend()
ax2.plot(accuracies)
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Accuracy")
plt.savefig("metrics.png")
plt.show()

In [ ]:
visualize_attention(model,'MNIST', "attention.png")